# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities = pd.read_csv("../output_data/cities.csv")
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"]
maxhumidity = humidity.max()

In [12]:
humidity_heatmap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity, point_radius=4)
humidity_heatmap.add_layer(heat_layer)
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
filtered_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0), :]
filtered_df = filtered_df.dropna(how='any')
filtered_df.reset_index(inplace=True)
del filtered_df['index']
filtered_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotels = []

for i in range(len(filtered_df)):
    lat = filtered_df.loc[i]['Lat']
    lng = filtered_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params = params)
    jsn = requested.json()
    
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
        
filtered_df["Hotel Name"] = hotels
filtered_df = filtered_df.dropna(how = 'any')
filtered_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in filtered_df.iterrows()]
locations = filtered_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
humidity_heatmap.add_layer(markers)

# Display Map
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))